# Data Gathering for Power Analysis

We've previously done power analyses for the Growth Team based on registration data from 2018. In this case, we want an updated analysis with more wikis, in order to understand how expanding to additional wikis will impact our statistical power.

This work is tracked in [T250120](https://phabricator.wikimedia.org/T250120).

The task asks to compare four groups of wikis:

* Original target wikis: Czech, Korean, Arabic, Vietnamese
* Current set: Czech, Korean, Arabic, Vietnamese, Ukrainian, Hungarian, Armenian, Basque
* Adding just French: Czech, Korean, Arabic, Vietnamese, Ukrainian, Hungarian, Armenian, Basque, French
* Adding our next set: Czech, Korean, Arabic, Vietnamese, Ukrainian, Hungarian, Armenian, Basque, French, Polish, Persian, Swedish, Danish, Indonesian, Italian, Portuguese.

The first part of this is to gather registration, activation, and retention numbers for those wikis, for both desktop and mobile registrations.

In [1]:
import datetime as dt

import pandas as pd
import numpy as np

from wmfdata import hive

In [8]:
## Configuration variables

## Original target, then the next four, then the additional set
wikis = ['cswiki', 'kowiki', 'viwiki', 'arwiki',
         'ukwiki', 'huwiki', 'hywiki', 'euwiki',
         'frwiki', 'plwiki', 'fawiki', 'svwiki',
         'dawiki', 'idwiki', 'itwiki', 'ptwiki']

## Activity tends to follow a yearly cycle, so let's use that.
start_date = '2019-01-01'
end_date = '2020-01-01'

## The mediawiki_history snapshot that we'll be using
snapshot = '2020-03'

In [13]:
activity_query = '''
WITH regs AS (
    SELECT wiki_db, event_user_id,
           date_format(event_user_creation_timestamp, "yyyy-MM-01") as reg_month
    FROM wmf.mediawiki_history
    WHERE snapshot = "{snapshot}"
    AND event_entity = "user"
    AND event_type = "create"
    AND event_user_is_created_by_self = TRUE
    AND size(event_user_is_bot_by_historical) = 0
    AND wiki_db IN ({wiki_list})
    AND event_user_creation_timestamp >= "{start_time}"
    AND event_user_creation_timestamp < "{end_time}"
),
mobile_data AS (
    SELECT wiki AS wiki_db,
           event.userid AS user_id,
           IF(event.displaymobile, 'mobile', 'desktop') AS platform
    FROM event_sanitized.serversideaccountcreation
    WHERE year = 2019
    AND wiki IN ({wiki_list})
    AND event.isselfmade = true
),
edits AS (
    SELECT wiki_db, event_user_id,
    SUM(IF(unix_timestamp(event_timestamp) -
           unix_timestamp(event_user_creation_timestamp) < 86400 , 1, 0)) AS activation_edits,
    SUM(IF(unix_timestamp(event_timestamp) - unix_timestamp(event_user_creation_timestamp)
           BETWEEN 86400 AND 15*86400, 1, 0)) AS retention_edits
    FROM wmf.mediawiki_history
    WHERE snapshot = "{snapshot}"
    AND event_entity = "revision"
    AND event_type = "create"
    AND wiki_db IN ({wiki_list})
    AND event_user_creation_timestamp >= "{start_time}"
    AND event_user_creation_timestamp < "{end_time}"
    AND SIZE(event_user_is_bot_by_historical) = 0
    GROUP BY wiki_db, event_user_id
)
SELECT regs.wiki_db,
       regs.event_user_id AS user_id,
       regs.reg_month,
       mobile_data.platform,
       coalesce(edits.activation_edits, 0) AS activation_edits,
       coalesce(edits.retention_edits, 0) AS retention_edits
FROM regs
JOIN mobile_data
ON regs.wiki_db = mobile_data.wiki_db
AND regs.event_user_id = mobile_data.user_id
LEFT JOIN edits
ON regs.wiki_db = edits.wiki_db
AND regs.event_user_id = edits.event_user_id
'''

Grab user activity data:

In [14]:
user_activity = hive.run(activity_query.format(
    snapshot = snapshot,
    wiki_list = ','.join('"{}"'.format(w) for w in wikis),
    start_time = start_date,
    end_time = end_date
))

Add boolean flags for whether a user is activated or retained:

In [16]:
user_activity['is_activated'] = user_activity['activation_edits'] > 0
user_activity['is_retained'] = user_activity['is_activated'] & (user_activity['retention_edits'] > 0)

Aggregate per wiki, platform, and month of registration counts of registrations, activations, and retentions:

In [23]:
registrations_agg = (user_activity.groupby(['wiki_db', 'reg_month', 'platform'])
                     .agg({'user_id' : 'count'})
                     .rename(columns = {'user_id' : 'n_registered'}))

In [24]:
activations_agg = (user_activity.loc[user_activity['is_activated'] == True]
                   .groupby(['wiki_db', 'reg_month', 'platform'])
                   .agg({'user_id' : 'count'})
                   .rename(columns = {'user_id' : 'n_activated'}))

In [25]:
retentions_agg = (user_activity.loc[user_activity['is_retained'] == True]
                  .groupby(['wiki_db', 'reg_month', 'platform'])
                  .agg({'user_id' : 'count'})
                  .rename(columns = {'user_id' : 'n_retained'}))

Merge the three aggregations to combine:

In [37]:
full_data = (registrations_agg.merge(activations_agg, how = 'left', left_index = True, right_index = True)
             .merge(retentions_agg, how = 'left', left_index = True, right_index = True)
             .fillna(0).reset_index())

In [38]:
## Calculate activation and retention proportions

full_data['prop_activated'] = full_data['n_activated'] / full_data['n_registered']
full_data['prop_retained'] = full_data['n_retained'] / full_data['n_activated']

Aggregate over the whole year and calculate monthly averages:

In [41]:
fullyear_agg = (full_data.groupby(['wiki_db', 'platform'])
                .agg({'n_registered' : 'mean', 'n_activated' : 'mean', 'n_retained' : 'mean',
                      'prop_activated' : 'mean', 'prop_retained' : 'mean'})
                .reset_index())

Write the resulting dataframe out as a TSV for import into R.

In [44]:
fullyear_agg.to_csv('datasets/aggregate_statistics.tsv',
                    header = True, index = False, sep = '\t')